# Desafio 1: Determinantes del ingreso

## Hito 1

### Preliminares

El objetivo de este ejercicio es desarrollar un modelo predictivo sobre la probabilidad de que un individuo presente salarios por sobre o bajo los 50000 dolares al año, en base a una serie de atributos socioeconomicos.

El desafio trata de un problema de **Clasificacion** para lo cual utilizare la ***Regresion Logistica***, y para validar el modelo me basare en las metricas Accuaracy, Precision y Recall.

### Aspectos Computacionales

A continuacion las librerias a utilizar seran:
   
    - warning: Para tener un notebook mas limpio y sin las alertas de deprecaion.
    - pandas: Libreria utilizada para el analisis de datos y lectura de archivos.
    - numpy: Libreria optimizada para el analisis de datos.
    - modulo stats: Modulo stats para realizar simulacion.
    - matplotlib: Libreria utilizada para realizar graficos
    - seborn : Libreria utilizada para realizar graficos de manera simple.
    - statsmodels: Liberia utilizada para realizar modelos.
    - func_analitics: Son las funciones creadas por mi para realizar la manipulacion de los datos.
    

In [1]:
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
import pandas as pd
import numpy as np
from scipy import stats
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
import statsmodels.formula.api as smf
import func_analitics as fa #funciones de analisis

In [2]:
df_origin = pd.read_csv('income-db.csv')

In [3]:
df = df_origin.copy()

In [4]:
df

age     workclass  fnlwgt     education  educational-num  \
0       25       Private  226802          11th                7   
1       38       Private   89814       HS-grad                9   
2       28     Local-gov  336951    Assoc-acdm               12   
3       44       Private  160323  Some-college               10   
4       18             ?  103497  Some-college               10   
...    ...           ...     ...           ...              ...   
48837   27       Private  257302    Assoc-acdm               12   
48838   40       Private  154374       HS-grad                9   
48839   58       Private  151910       HS-grad                9   
48840   22       Private  201490       HS-grad                9   
48841   52  Self-emp-inc  287927       HS-grad                9   

           marital-status         occupation relationship   race  gender  \
0           Never-married  Machine-op-inspct    Own-child  Black    Male   
1      Married-civ-spouse    Farming-fishing      Husband  White    Male   
2      Married-civ-spouse    Protective-serv      Husband  White    Male   
3      Married-civ-spouse  Machine-op-inspct      Husband  Black    Male   
4           Never-married                  ?    Own-child  White  Female   
...                   ...                ...          ...    ...     ...   
48837  Married-civ-spouse       Tech-support         Wife  White  Female   
48838  Married-civ-spouse  Machine-op-inspct      Husband  White    Male   
48839             Widowed       Adm-clerical    Unmarried  White  Female   
48840       Never-married       Adm-clerical    Own-child  White    Male   
48841  Married-civ-spouse    Exec-managerial         Wife  White  Female   

       capital-gain  capital-loss  hours-per-week native-country income  
0                 0             0              40  United-States  <=50K  
1                 0             0              50  United-States  <=50K  
2                 0             0              40  United-States   >50K  
3              7688             0              40  United-States   >50K  
4                 0             0              30  United-States  <=50K  
...             ...           ...             ...            ...    ...  
48837             0             0              38  United-States  <=50K  
48838             0             0              40  United-States   >50K  
48839             0             0              40  United-States  <=50K  
48840             0             0              20  United-States  <=50K  
48841         15024             0              40  United-States   >50K  

[48842 rows x 15 columns]

### A continuacion realizare la recodificacion de las variables.

In [5]:
df.occupation.value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
?                    2809
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [6]:
for columna in df.columns:
    df[columna] = np.where(df[columna]=='?', np.nan, df[columna])

In [7]:
df.occupation.value_counts()

Prof-specialty       6172
Craft-repair         6112
Exec-managerial      6086
Adm-clerical         5611
Sales                5504
Other-service        4923
Machine-op-inspct    3022
Transport-moving     2355
Handlers-cleaners    2072
Farming-fishing      1490
Tech-support         1446
Protective-serv       983
Priv-house-serv       242
Armed-Forces           15
Name: occupation, dtype: int64

In [8]:
elem=['Prof-specialty', 'Exec-managerial', 'Adm-clerical','Sales, Tech-support']

In [9]:
df.loc[(df['occupation'] == 'Prof-specialty') | (df['occupation'] =='Exec-managerial') | 
       (df['occupation'] == 'Adm-clerical') | (df['occupation'] =='Sales') | 
       (df['occupation'] =='Tech-support'),'collars'] = 'white-collar'

In [10]:
df.loc[(df['occupation'] == 'Craft-repair') | (df['occupation'] =='Machine-op-inspct') | 
       (df['occupation'] == 'Transport-moving,') | (df['occupation'] =='Handlers-cleaners') | 
       (df['occupation'] =='Farming-fishing') | (df['occupation'] =='Protective-serv') | 
       (df['occupation'] =='Priv-house-serv'),'collars'] = 'blue-collar'

In [11]:
df.loc[(df['occupation'] == 'Other-service') | (df['occupation'] =='Armed-Forces'),'collars'] = 'others'

In [12]:
df.collars.value_counts()

white-collar    24819
blue-collar     13921
others           4938
Name: collars, dtype: int64

In [13]:
cond_workclass = [(df['workclass']=='Federal-gov'),(df['workclass']=='State-gov') | (df['workclass']=='Local-gov'),
                  (df['workclass']=='Self-emp-inc') | (df['workclass']=='Self-emp-not-inc'), 
                  (df['workclass']=='Never-worked')| (df['workclass']=='Without-pay'), 
                  (df['workclass']=='Private')]
opt_workclass = ['federal-gov', 'state-level-gov', 'self-employed', 'unemployed', 'private']

df['workclass_recod']=np.select(cond_workclass,opt_workclass)

In [14]:
df.workclass_recod.value_counts()

private            33906
self-employed       5557
state-level-gov     5117
0                   2799
federal-gov         1432
unemployed            31
Name: workclass_recod, dtype: int64

In [15]:
cond_education = [df['education'] == 'Preschool', (df['education'] == '1st-4th') | (df['education'] == '5th-6th'),
                 (df['education'] == '7th-8th') | (df['education'] =='9th') | (df['education'] =='10th') | 
                  (df['education'] =='11th') | (df['education'] =='12th') |(df['education'] =='HS-grad'),
                 (df['education'] == 'Assoc-voc') | (df['education'] =='Assoc-acdm') | 
                  (df['education'] == 'Some-college'), (df['education'] == 'Bachelors') | 
                  (df['education'] == 'Masters') | (df['education'] == 'Prof-school') | 
                  (df['education'] == 'Doctorate')]
opt_education = ['preschool', 'elementary-school','high-school', 'college', 'university']
df['educ_recod'] = np.select(cond_education, opt_education)

In [16]:
cond_marital = [(df['marital-status'] == 'Married-civ-spouse')|(df['marital-status'] == 'Married-spouse-absent')|
                (df['marital-status'] == 'Married-AF-spouse'), (df['marital-status'] == 'Divorced'), 
                (df['marital-status'] == 'Separated'), (df['marital-status'] == 'Widowed'), 
                (df['marital-status'] == 'Never-married')]
opt_marital = ['married', 'divorced', 'separated', 'widowed', 'Never-married']
df['civstatus']=np.select(cond_marital, opt_marital)

In [17]:
df.civstatus.value_counts()

married          23044
Never-married    16117
divorced          6633
separated         1530
widowed           1518
Name: civstatus, dtype: int64

In [18]:
df['marital-status'].value_counts()

Married-civ-spouse       22379
Never-married            16117
Divorced                  6633
Separated                 1530
Widowed                   1518
Married-spouse-absent      628
Married-AF-spouse           37
Name: marital-status, dtype: int64

In [19]:
df.income = np.where(df['income']=='>50K', 1, 0)

In [20]:
df = df.replace(['United-States', 'Puerto-Rico', 'Canada', 'Cuba', 'Honduras', 'Mexico',
                              'Dominican-Republic', 'Jamaica', 'Ecuador', 'Haiti', 'Columbia', 'Guatemala', 
                              'Nicaragua', 'El-Salvador', 'Trinadad&Tobago', 'Peru'], 'america') 
df = df.replace(['England','Germany', 'Greece', 'Poland', 'France', 'Hungary', 'Holand-Netherlands', 'Yugoslavia',
           'Scotland', 'Ireland', 'Portugal', 'Italy'], 'europe')
df = df.replace(['Outlying-US(Guam-USVI-etc)'], 'oceania')
df = df.replace(['Cambodia', 'India', 'Japan', 'South', 'China', 'Iran', 'Philippines', 'Vietnam', 'Laos',
            'Taiwan', 'Thailand', 'Hong'], 'asia')

In [21]:
df

age     workclass    fnlwgt     education  educational-num  \
0      25.0       Private  226802.0          11th              7.0   
1      38.0       Private   89814.0       HS-grad              9.0   
2      28.0     Local-gov  336951.0    Assoc-acdm             12.0   
3      44.0       Private  160323.0  Some-college             10.0   
4      18.0           NaN  103497.0  Some-college             10.0   
...     ...           ...       ...           ...              ...   
48837  27.0       Private  257302.0    Assoc-acdm             12.0   
48838  40.0       Private  154374.0       HS-grad              9.0   
48839  58.0       Private  151910.0       HS-grad              9.0   
48840  22.0       Private  201490.0       HS-grad              9.0   
48841  52.0  Self-emp-inc  287927.0       HS-grad              9.0   

           marital-status         occupation relationship   race  gender  \
0           Never-married  Machine-op-inspct    Own-child  Black    Male   
1      Married-civ-spouse    Farming-fishing      Husband  White    Male   
2      Married-civ-spouse    Protective-serv      Husband  White    Male   
3      Married-civ-spouse  Machine-op-inspct      Husband  Black    Male   
4           Never-married                NaN    Own-child  White  Female   
...                   ...                ...          ...    ...     ...   
48837  Married-civ-spouse       Tech-support         Wife  White  Female   
48838  Married-civ-spouse  Machine-op-inspct      Husband  White    Male   
48839             Widowed       Adm-clerical    Unmarried  White  Female   
48840       Never-married       Adm-clerical    Own-child  White    Male   
48841  Married-civ-spouse    Exec-managerial         Wife  White  Female   

       capital-gain  capital-loss  hours-per-week native-country  income  \
0               0.0           0.0            40.0        america       0   
1               0.0           0.0            50.0        america       0   
2               0.0           0.0            40.0        america       1   
3            7688.0           0.0            40.0        america       1   
4               0.0           0.0            30.0        america       0   
...             ...           ...             ...            ...     ...   
48837           0.0           0.0            38.0        america       0   
48838           0.0           0.0            40.0        america       1   
48839           0.0           0.0            40.0        america       0   
48840           0.0           0.0            20.0        america       0   
48841       15024.0           0.0            40.0        america       1   

            collars  workclass_recod   educ_recod      civstatus  
0       blue-collar          private  high-school  Never-married  
1       blue-collar          private  high-school        married  
2       blue-collar  state-level-gov      college        married  
3       blue-collar          private      college        married  
4               NaN                0      college  Never-married  
...             ...              ...          ...            ...  
48837  white-collar          private      college        married  
48838   blue-collar          private  high-school        married  
48839  white-collar          private  high-school        widowed  
48840  white-collar          private  high-school  Never-married  
48841  white-collar    self-employed  high-school        married  

[48842 rows x 19 columns]

In [22]:
df['native-country'].value_counts()

america    46086
asia        1096
europe       780
oceania       23
Name: native-country, dtype: int64

In [23]:
df=df.drop(['occupation', 'education', 'workclass','marital-status'], axis = 1)

In [24]:
df = df.rename(columns = {'native-country':'region'})

### Tabla con los datos recodificados

In [25]:
df

age    fnlwgt  educational-num relationship   race  gender  \
0      25.0  226802.0              7.0    Own-child  Black    Male   
1      38.0   89814.0              9.0      Husband  White    Male   
2      28.0  336951.0             12.0      Husband  White    Male   
3      44.0  160323.0             10.0      Husband  Black    Male   
4      18.0  103497.0             10.0    Own-child  White  Female   
...     ...       ...              ...          ...    ...     ...   
48837  27.0  257302.0             12.0         Wife  White  Female   
48838  40.0  154374.0              9.0      Husband  White    Male   
48839  58.0  151910.0              9.0    Unmarried  White  Female   
48840  22.0  201490.0              9.0    Own-child  White    Male   
48841  52.0  287927.0              9.0         Wife  White  Female   

       capital-gain  capital-loss  hours-per-week   region  income  \
0               0.0           0.0            40.0  america       0   
1               0.0           0.0            50.0  america       0   
2               0.0           0.0            40.0  america       1   
3            7688.0           0.0            40.0  america       1   
4               0.0           0.0            30.0  america       0   
...             ...           ...             ...      ...     ...   
48837           0.0           0.0            38.0  america       0   
48838           0.0           0.0            40.0  america       1   
48839           0.0           0.0            40.0  america       0   
48840           0.0           0.0            20.0  america       0   
48841       15024.0           0.0            40.0  america       1   

            collars  workclass_recod   educ_recod      civstatus  
0       blue-collar          private  high-school  Never-married  
1       blue-collar          private  high-school        married  
2       blue-collar  state-level-gov      college        married  
3       blue-collar          private      college        married  
4               NaN                0      college  Never-married  
...             ...              ...          ...            ...  
48837  white-collar          private      college        married  
48838   blue-collar          private  high-school        married  
48839  white-collar          private  high-school        widowed  
48840  white-collar          private  high-school  Never-married  
48841  white-collar    self-employed  high-school        married  

[48842 rows x 15 columns]